In [1]:
import pandas as pd
from math import sqrt
import glob
from pathlib import Path
import pandas_profiling
from matplotlib import pyplot
import seaborn as sns
from pandas.plotting import autocorrelation_plot
from datetime import datetime
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from math import sqrt

In [2]:
pd.set_option('display.max_rows', 1000)

In [3]:
df = pd.read_csv('processed_dataset.csv')

In [4]:
df['measure_date'] = pd.to_datetime(pd.to_datetime(df['measure_date']).dt.date)
df['departure_date'] = pd.to_datetime(pd.to_datetime(df['departure_date']).dt.date)
df['arrival_date'] = pd.to_datetime(pd.to_datetime(df['arrival_date']).dt.date)

In [5]:
df.head()

,departure_city,arrival_city,flight_duration,carrier_name,agent_name,flight_number,price,measure_date,departure_date,departure_hour,departure_minute,arrival_date,arrival_hour,arrival_minute,days_left_to_departure,previous_price_for_trip
0,moscow,budapest,165,aeroflot,aeroflot,2468,3710.37,2020-03-01,2020-05-01,20,10,2020-05-01,21,55,61,NaN
1,moscow,budapest,155,wizz air,wizz air,2490,4763.70,2020-03-01,2020-05-01,10,5,2020-05-01,11,40,61,NaN
2,moscow,budapest,130,lot,lot,5676,5306.59,2020-03-01,2020-05-01,18,5,2020-05-01,19,15,61,NaN
3,moscow,budapest,220,brussels airlines,brussels airlines,2846,1879.37,2020-03-01,2020-05-04,15,45,2020-05-04,18,25,64,NaN
4,moscow,budapest,155,wizz air,wizz air,2490,2177.55,2020-03-01,2020-05-04,10,5,2020-05-04,11,40,64,NaN


In [6]:
df.dtypes

departure_city                     object
arrival_city                       object
flight_duration                     int64
carrier_name                       object
agent_name                         object
flight_number                       int64
price                             float64
measure_date               datetime64[ns]
departure_date             datetime64[ns]
departure_hour                      int64
departure_minute                    int64
arrival_date               datetime64[ns]
arrival_hour                        int64
arrival_minute                      int64
days_left_to_departure              int64
previous_price_for_trip           float64
dtype: object

In [7]:
df['measure_date'].max()-df['measure_date'].min()

Timedelta('27 days 00:00:00')

In [8]:
tscv = TimeSeriesSplit(n_splits=3)

In [9]:
def rmse(real, predicted):
    return sqrt(mean_squared_error(real, predicted))

In [23]:
def fit_transform_previous(train, test):
    predicted = train.iloc[-1]['price']
    predict_len = len(test)
    return pd.Series([predicted]).repeat(predict_len)

In [24]:
def fit_transform_average(train, test):
    predicted = train['price'].mean()
    predict_len = len(test)
    return pd.Series([predicted]).repeat(predict_len)

In [25]:
cv_rmse = []
for idx, split_idx_tuple in enumerate(tscv.split(df)):
    x_train_idx, x_test_idx = split_idx_tuple[0], split_idx_tuple[1]
    train, test = df.iloc[x_train_idx], df.iloc[x_test_idx]
    prediction = fit_transform_previous(train, test)
    current_rmse = rmse(prediction, test['price'])
    cv_rmse.append(current_rmse)
    print("MSE on {}th fold: {}".format(idx, current_rmse))

mean_rmse = sum(cv_rmse) / len(cv_rmse)
print("mean rmse: {}".format(mean_rmse))

MSE on 0th fold: 2900.171979035279
MSE on 1th fold: 3124.4506901320055
MSE on 2th fold: 3185.6516897343113
mean rmse: 3070.0914529671986


In [26]:
cv_rmse = []
for idx, split_idx_tuple in enumerate(tscv.split(df)):
    x_train_idx, x_test_idx = split_idx_tuple[0], split_idx_tuple[1]
    train, test = df.iloc[x_train_idx], df.iloc[x_test_idx]
    prediction = fit_transform_average(train, test)
    current_rmse = rmse(prediction, test['price'])
    cv_rmse.append(current_rmse)
    print("MSE on {}th fold: {}".format(idx, current_rmse))

mean_rmse = sum(cv_rmse) / len(cv_rmse)
print("mean rmse: {}".format(mean_rmse))

MSE on 0th fold: 2586.6157903039366
MSE on 1th fold: 2485.4311423963222
MSE on 2th fold: 2502.228901134096
mean rmse: 2524.7586112781187
